<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [107]:
import pandas as pd
import psycopg2
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [108]:
# connection parameters from config.txt file
params = {k: v.strip() for k, v in (l.split('=') for l in open('config.txt'))}

In [109]:
# connect database
connection = psycopg2.connect(
    dbname = params['DBNAME'],
    user = params['USER'],
    host = params['HOST'],
    password = params['PASSWORD'],
    port = params['PORT']
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [110]:
query_3_1 = f'''
SELECT COUNT(v.id) vacancies_cnt
FROM public.vacancies v
'''

In [111]:
# read query as DataFrame
vacancies = pd.read_sql_query(query_3_1, connection)
display(vacancies)

/tmp/ipykernel_10232/663732383.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vacancies_cnt
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [112]:
query_3_2 = f'''
SELECT COUNT(e.id) employers_cnt
FROM public.employers e
'''

In [113]:
employers_cnt = pd.read_sql_query(query_3_2, connection)
display(employers_cnt)

/tmp/ipykernel_10232/3006259819.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employers_cnt
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [114]:
query_3_3 = f'''
SELECT COUNT(a.id) areas_cnt
FROM public.areas a
'''

In [115]:
areas_cnt = pd.read_sql_query(query_3_3, connection)
display(areas_cnt)

/tmp/ipykernel_10232/2993982255.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,areas_cnt
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [116]:
query_3_4 = f'''
SELECT COUNT(i.id) industries_cnt
FROM public.industries i
'''

In [117]:
industries_cnt = pd.read_sql_query(query_3_4, connection)
display(industries_cnt)

/tmp/ipykernel_10232/4202181286.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,industries_cnt
0,294


***

Минимальное, среднее, максимальное значения нижней границы зарплаты, там где она указана:

In [118]:
# Filter not null entries

query_3_extra1 = f'''
SELECT MIN(v.salary_from) min_lower_salary,
       AVG(v.salary_from) avg_lower_salary,
       MAX(v.salary_from) max_lower_salary
FROM public.vacancies v
WHERE v.salary_from IS NOT NULL
'''

In [119]:
lower_salary = pd.read_sql_query(query_3_extra1, connection)
display(lower_salary)

/tmp/ipykernel_10232/1251855214.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,min_lower_salary,avg_lower_salary,max_lower_salary
0,30,71064.657901,1000000


Минимальное, среднее, максимальное значения верхней границы зарплаты, там где она указана:

In [120]:
query_3_extra2 = f'''
SELECT MIN(v.salary_to) min_upper_salary,
       AVG(v.salary_to) avg_upper_salary,
       MAX(v.salary_to) max_upper_salary
FROM public.vacancies v
WHERE v.salary_to IS NOT NULL
'''

In [121]:
upper_salary = pd.read_sql_query(query_3_extra2, connection)
display(upper_salary)

/tmp/ipykernel_10232/1513011084.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,min_upper_salary,avg_upper_salary,max_upper_salary
0,60,110536.741923,1000000


Вывод экстремальных значений заработной платы:

In [122]:
query_3_extra3 = f'''
SELECT e.name employer,
       v.name vacancy,
       a.name area,
       v.salary_from lower_salary,
       v.salary_to upper_salary
FROM public.vacancies v
     JOIN public.employers e ON v.employer_id = e.id
     JOIN public.areas a ON v.area_id = a.id
WHERE (v.salary_from = 30) OR
      (v.salary_from = 1000000) OR
      (v.salary_to = 60) OR
      (v.salary_to = 1000000)
'''

In [123]:
salary_extreme = pd.read_sql_query(query_3_extra3, connection)
display(salary_extreme)

/tmp/ipykernel_10232/45592727.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer,vacancy,area,lower_salary,upper_salary
0,Alpha Personnel,Технический директор,Санкт-Петербург,1000000,NaN
1,iBEC Systems,QA Automation Engineer,Алматы,400000,1000000.0
2,Шеф Маркет,СТО/Директор по разработке,Москва,500000,1000000.0
3,Группа компаний Империя,Помощник IT-специалиста,Сочи,30,NaN
4,Агентство недвижимости ЛЕТО,Брокер по продаже апартаментов,Сочи,500000,1000000.0
5,РЕФОРС,Системный администратор,Челябинск,30,50000.0
6,Альтаир,Database Administrator (Администратор баз данных),Смоленск,60,60.0
7,Группа компаний Империя,Помощник IT-специалиста,Сочи,30,NaN


Среднее число вакансий на работодателя:

In [124]:
round(vacancies['vacancies_cnt'][0] / employers_cnt['employers_cnt'][0], 1)

2.1

Среднее число работодателей на регион:

In [125]:
round(employers_cnt['employers_cnt'][0] / areas_cnt['areas_cnt'][0], 1)

17.3

Среднее число работодателей на сферу деятельности:

In [126]:
round(employers_cnt['employers_cnt'][0] / industries_cnt['industries_cnt'][0], 1)

79.9

***

*В среднем наблюдаются $2.1$ вакансии на работодателя, $17.3$ работодателя на регион, $79.9$ работодателей на сферу деятельности. Такие результаты могут быть связаны с тем, что большинство работодателей - небольшие компании с малым количеством вакансий.*

*Такое количество работодателей на регион может наблюдаться из-за того, что существует вложенность регионов (один регион включает в себя другой), а также из-за наличия малонаселённых регионов с небольшим количеством работодателей.*

*Количество работодателей на сферу деятельности может быть вызвано тем, что далеко не все работодатели её указывают.*

*Анализ экстремальных значений заработной платы показывает, что аномально низкие граничные значения зарплаты связаны с опечатками (при вводе данных указали значения в тысячах рублей).*

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [127]:
top_area_vac = 5 # first five areas by vacancies count

query_4_1 = f'''
SELECT a.name area,
       COUNT(v.id) cnt
FROM public.vacancies v 
     JOIN public.areas a ON v.area_id = a.id
GROUP BY a.name
ORDER BY cnt DESC
LIMIT {top_area_vac}
'''

In [128]:
area_vacancies = pd.read_sql_query(query_4_1, connection)
display(area_vacancies)

/tmp/ipykernel_10232/4191533609.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


In [129]:
# Create 'Другие' record with sum of the other records

query_4_1a = f''' 
with n_last as (
     SELECT a.name area,
            COUNT(v.id) cnt
     FROM public.vacancies v 
          JOIN public.areas a ON v.area_id = a.id
     GROUP BY a.name
     ORDER BY cnt DESC
     OFFSET {top_area_vac}
)

(SELECT a.name area,
       COUNT(v.id) cnt
FROM public.vacancies v 
     JOIN public.areas a ON v.area_id = a.id
GROUP BY a.name
ORDER BY cnt DESC
LIMIT {top_area_vac})

UNION ALL
               
(SELECT 'Другие' area,
        SUM(nl.cnt) cnt
FROM n_last nl
)
'''

In [130]:
area_vac_percentage = pd.read_sql_query(query_4_1a, connection)
area_vac_percentage['cnt'] = area_vac_percentage['cnt'].astype('int64')
display(area_vac_percentage)

/tmp/ipykernel_10232/1473061825.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Другие,35003


In [131]:
fig = px.pie(
    area_vac_percentage,
    values=area_vac_percentage['cnt'],
    names=area_vac_percentage['area'],
    title=f'Топ-{top_area_vac} регионов по числу вакансий',
    height=600,
    width=1000
)
fig.update_traces(textposition='outside', textinfo='percent+label', textfont_size=14)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/top_areas_vac.html')
fig.write_image('images/top_areas_vac.png')

![Top regions by vacancies](images/top_areas_vac.png)

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [132]:
query_4_2 = f'''
SELECT COUNT(v.id) cnt
FROM public.vacancies v 
WHERE (v.salary_from IS NOT NULL) OR (v.salary_to IS NOT NULL)
'''

In [133]:
salary_not_null = pd.read_sql_query(query_4_2, connection)
display(salary_not_null)

/tmp/ipykernel_10232/2649882509.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,cnt
0,24073


Доля вакансий в процентах, у которых заполнено хотя бы одно из двух полей с зарплатой:

In [134]:
round(salary_not_null['cnt'][0] / vacancies['vacancies_cnt'][0] * 100, 1)

48.9

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [135]:
# ROUND function to round the result

query_4_3 = f'''
SELECT ROUND(AVG(v.salary_from)) mean_lower_salary,
       ROUND(AVG(v.salary_to)) mean_upper_salary
FROM public.vacancies v
'''

In [136]:
mean_salary = pd.read_sql_query(query_4_3, connection)
display(mean_salary)

/tmp/ipykernel_10232/3026739217.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,mean_lower_salary,mean_upper_salary
0,71065.0,110537.0


График границ заработной платы (пунктирной линией обозначены соответствующие средние значения):

In [137]:
query_4_3a = f'''
(SELECT 'lower' salary_type,
        v.salary_from salary
FROM public.vacancies v
WHERE v.salary_from is NOT NULL
ORDER BY salary)

UNION ALL

(SELECT 'upper' salary_type,
       v.salary_to salary
FROM public.vacancies v
WHERE v.salary_to IS NOT NULL
ORDER BY salary)
'''

salary_distr = pd.read_sql_query(query_4_3a, connection)

fig0 = px.box(
    salary_distr,
    x='salary',
    color='salary_type',
    points='outliers',
    height=600,
    width=1000,
    title='Распределение границ заработной платы'
)
fig0.add_vline(mean_salary['mean_lower_salary'][0], line_width=2, line_dash="dash", line_color="blue")
fig0.add_vline(mean_salary['mean_upper_salary'][0], line_width=2, line_dash="dash", line_color="red")
fig0.update_yaxes(minor_showgrid=True)
fig0.layout.xaxis.title.text = 'Заработная плата, руб'
fig0.layout.yaxis.title.text = 'Тип границы заработной платы'
fig0.show()

fig0.write_html("images/salary_box.html")

/tmp/ipykernel_10232/3874728299.py:17: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [138]:
query_4_4 = f'''
SELECT v.schedule schedule, 
       v.employment employment, 
       COUNT(v.id) vacancies_cnt
FROM public.vacancies v
GROUP BY v.schedule, v.employment
ORDER BY vacancies_cnt DESC
'''

In [139]:
schedule_cnt = pd.read_sql_query(query_4_4, connection)
display(schedule_cnt)

/tmp/ipykernel_10232/1605552346.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,vacancies_cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


Самые распространённые формы занятости:

In [140]:
top_schedule = 5  # number of most popular schedules

# Count vacancies via performing a subquery
query_4_4a = f''' 
with n_last as (
SELECT v.schedule schedule, 
       v.employment employment, 
       COUNT(v.id) vacancies_cnt
FROM public.vacancies v
GROUP BY v.schedule, v.employment
ORDER BY vacancies_cnt DESC
OFFSET {top_schedule})

(SELECT v.schedule schedule, 
       v.employment employment, 
       COUNT(v.id) vacancies_cnt
FROM public.vacancies v
GROUP BY v.schedule, v.employment
ORDER BY vacancies_cnt DESC
LIMIT {top_schedule})

UNION ALL
               
(SELECT 'Другие' schedule,
        '' employment,
        SUM(nl.vacancies_cnt) vacancies_cnt
FROM n_last nl
)
'''

schedule_topn = pd.read_sql_query(query_4_4a, connection)
display(schedule_topn)

/tmp/ipykernel_10232/4076628089.py:31: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,vacancies_cnt
0,Полный день,Полная занятость,35367.0
1,Удаленная работа,Полная занятость,7802.0
2,Гибкий график,Полная занятость,1593.0
3,Удаленная работа,Частичная занятость,1312.0
4,Сменный график,Полная занятость,940.0
5,Другие,,2183.0


In [141]:
fig = px.pie(
    schedule_topn,
    values=schedule_topn['vacancies_cnt'],
    names=schedule_topn['schedule'] + ' - ' + schedule_topn['employment'],
    title=f'Топ-{top_schedule} форм занятости по числу вакансий',
    height=600,
    width=1000
)
fig.update_traces(textposition='outside', textinfo='percent+label', textfont_size=14)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/top_schedule.html')
fig.write_image('images/top_schedule.png')

![Top schedules by salaries](./images/top_schedule.png)

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [142]:
query_4_5 = f'''
SELECT v.experience experience, 
       COUNT(v.id) vacancies_cnt
FROM public.vacancies v
GROUP BY v.experience
ORDER BY vacancies_cnt
'''

In [143]:
experience = pd.read_sql_query(query_4_5, connection)
display(experience)

/tmp/ipykernel_10232/3307137788.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,vacancies_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [144]:
fig = px.pie(
    experience,
    values=experience['vacancies_cnt'],
    names=experience['experience'],
    title='Доля вакансий в зависимости от опыта работы',
    height=600,
    width=1000
)
fig.update_traces(textposition='outside', textinfo='percent+label', textfont_size=14)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/vac_experience.html')
fig.write_image('images/vac_experience.png')

![Vacancies by experience](./images/vac_experience.png)

***

Первые пять регионов с наивысшими средними зарплатами:

In [145]:
region_top_salary = 5 # number of regions with highest mean salary

query_4_extra1 = f'''
SELECT a.name area,
       AVG(v.salary_from) mean_lower_salary,
       AVG(v.salary_to) mean_upper_salary
FROM public.vacancies v
     LEFT JOIN public.areas a ON v.area_id = a.id
WHERE v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL
GROUP BY a.name
ORDER BY mean_upper_salary DESC, mean_lower_salary DESC
LIMIT {region_top_salary}
'''

In [146]:
area_salaries = pd.read_sql_query(query_4_extra1, connection)
display(area_salaries)

/tmp/ipykernel_10232/1426367140.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,mean_lower_salary,mean_upper_salary
0,Германия,348270.0,557232.0
1,Кутаиси,160733.5,350691.0
2,Черногория,282501.0,331208.0
3,Болгария,200000.0,300000.0
4,Кинешма,150000.0,300000.0


Выведем количество вакансий, минимальную и максимальную зарплату в этих регионах:

In [147]:
area_top_salary = tuple(area_salaries['area'])

query_4_extra2 = f'''
SELECT a.name area,
       COUNT(v.id) vacancies_count,
       MIN(v.salary_from) min_salary,
       MAX(v.salary_to) max_salary
FROM public.vacancies v
     LEFT JOIN public.areas a ON v.area_id = a.id
WHERE a.name IN {area_top_salary}
GROUP BY a.name
ORDER BY vacancies_count DESC
'''

In [148]:
location_salaries = pd.read_sql_query(query_4_extra2, connection)
display(location_salaries)

/tmp/ipykernel_10232/2546420116.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,vacancies_count,min_salary,max_salary
0,Германия,19,348270,557232
1,Черногория,14,233794,350691
2,Болгария,12,200000,450000
3,Кутаиси,4,87673,409140
4,Кинешма,3,100000,300000


Рассмотрим подробнее информацию о вакансиях в Кутаиси и Кинешме:

In [149]:
query_4_extra3 = f'''
SELECT a.name area,
       v.name vacancy,
       v.salary_from,
       v.salary_to
FROM public.vacancies v
     LEFT JOIN public.areas a ON v.area_id = a.id
WHERE a.name IN ('Кутаиси', 'Кинешма')
ORDER BY area
'''

In [150]:
location_vacancies = pd.read_sql_query(query_4_extra3, connection)
display(location_vacancies)

/tmp/ipykernel_10232/2164735262.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,vacancy,salary_from,salary_to
0,Кинешма,Java-разработчик,150000.0,300000.0
1,Кинешма,Разработчик 1С,100000.0,NaN
2,Кинешма,Инженер технической поддержки (подработка),NaN,NaN
3,Кутаиси,Unity Developer in a start-up game studio,87673.0,292242.0
4,Кутаиси,Android разработчик,NaN,NaN
5,Кутаиси,Senior Java Developer,233794.0,409140.0
6,Кутаиси,Ведущий программист 1C,150000.0,NaN


***

*По итогам исследований мы видим, что чуть менее чем $29\%$ всех вакансий сосредоточены в пяти мегаполисах. Это можно объяснить тем, что эти города - крупные экономические центры, с большим спросом на различного вида услуги.*

*Также было установлено, что лишь половина вакансий имеют хотя бы одну из границ заработной платы. Причины здесь могут быть различные. Среди них может быть и такая, что работодатель может рассматривать кандидатов с различным опытом, и в зависимости от этого может меняться и зарплата.*

*Были получены средние значения для нижней и верхней границ заработной платы соответственно: $71065$ и $110537$ руб. Эти значения выше медианных на $20-30$ тысяч руб из-за серии выбросов в виде высоких зарплат.*

*Почти $72\%$ всех вакансий требуют полный день и полную занятость. Факторы, которые могут приводить к такому результату: удалённая работа присутствует далеко не во всех сферах деятельности или, где присутствует частично, работодатель всё равно может указать в вакансии полный день. Сменный график, различные частичные формы занятости: такого персонала у представленных в базе работодателей много не требуется, или персонал может наняться в ряде случаев к кому-то ещё.*

*Более половины вакансий требуют опыт от 1 до 3 лет, чуть менее $30\%$ требуют от 3 до 6 лет. Примерно $15\%$ не требуют опыта совсем, и всего лишь в менее чем $3\%$ случаев желают видеть соискателей с опытом более 6 лет. Такие результаты могут быть обусловлены тем, что большинство сфер деятельности не требуют очень высокой квалификации, или специалистов с хорошим опытом в процентном соотношении не требуется в большом количестве в фирме, в сравнении со специалистами с более скромным опытом. Вакансии без опыта могут быть как стажировками, так и связаны с малоквалифицированным трудом. Малое количество вакансий с опытом работы более 6 лет может быть связано с тем, что персонал с такой квалификацией склонен не менять место работы, либо если уходить, то менять и сферу деятельности. Требования к заработной плате у таких специалистов, как правило, соответствующие.*

*Среди первых пяти регионов по средним зарплатам мы можем наблюдать как регионы из ЕС, так и другие регионы (Кутаиси, Кинешма). Наличие здесь регионов из ЕС может объясняться более высокими заработными платами в целом. Подробное рассмотрение вакансий из Кутаиси и Кинешмы выявляет малое число этих вакансий и то, что эти вакансии преимущественно из сферы информационных технологий. Таким образом, такие средние зарплаты там объясняются нерепрезентативностью выборки.*

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [151]:
place = 5 

query_5_1 = f'''
(
SELECT e.name employer,
       COUNT(v.id) vacancies_cnt
FROM public.vacancies v
     JOIN public.employers e ON v.employer_id = e.id
GROUP BY e.name
ORDER BY vacancies_cnt DESC
LIMIT 1)

UNION ALL 

(
SELECT e.name employer,
       COUNT(v.id) vacancies_cnt
FROM public.vacancies v
     JOIN public.employers e ON v.employer_id = e.id
GROUP BY e.name
ORDER BY vacancies_cnt DESC
OFFSET {place - 1}
LIMIT 1)
'''

In [152]:
employers_vacancies = pd.read_sql_query(query_5_1, connection)
display(employers_vacancies)

/tmp/ipykernel_10232/2755366348.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer,vacancies_cnt
0,Яндекс,1933
1,Газпром нефть,331


Работодатели с наибольшим количеством вакансий:

In [153]:
top_employers_vac = 5

# Apply window function to subquery to calculate percent
query_5_1a = f''' 
SELECT t1.employer employer,
       t1.vacancies_cnt vacancies_cnt,
       ROUND(100 * t1.vacancies_cnt / SUM(t1.vacancies_cnt) over (), 1) vacancies_percentage
FROM (
     SELECT e.name employer,
            COUNT(v.id) vacancies_cnt
     FROM public.vacancies v
          JOIN public.employers e ON v.employer_id = e.id
     GROUP BY e.name
     ORDER BY vacancies_cnt DESC
     ) as t1
LIMIT {top_employers_vac}
'''

employers_topn = pd.read_sql_query(query_5_1a, connection)
display(employers_topn)

/tmp/ipykernel_10232/2054028017.py:19: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer,vacancies_cnt,vacancies_percentage
0,Яндекс,1933,3.9
1,Ростелеком,491,1.0
2,Тинькофф,444,0.9
3,СБЕР,428,0.9
4,Газпром нефть,331,0.7


In [154]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(x=employers_topn['employer'], 
           y=employers_topn['vacancies_percentage'], 
           marker_color="MediumPurple",
           name="Процент"),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=employers_topn['employer'], 
           y=employers_topn['vacancies_cnt'], 
           marker_color="MediumPurple",
           name="Число"),
    secondary_y=True,
)

fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    title_text=f"Топ-{top_employers_vac} работодателей по числу вакансий"
)

fig.update_xaxes(title_text="Работодатель")

fig.update_yaxes(title_text="Процент от общего числа вакансий", secondary_y=False)
fig.update_yaxes(title_text="Количество вакансий", secondary_y=True)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/top_employer_vac.html')
fig.write_image('images/top_employer_vac.png')

![Top employes by vacancies](./images/top_employer_vac.png)

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [155]:
# Filter areas with zero vacancies
query_5_2 = f'''
SELECT a.name area,
       COUNT(e.id) employers_cnt,
       COUNT(v.id) vacancies_cnt
FROM public.areas a
     LEFT JOIN public.vacancies v ON a.id = v.area_id
     LEFT JOIN public.employers e ON a.id = e.area
GROUP BY a.name
HAVING COUNT(v.id) = 0
ORDER BY employers_cnt DESC
LIMIT 1
'''

In [156]:
area_zero_vacancies1 = pd.read_sql_query(query_5_2, connection)
display(area_zero_vacancies1)

/tmp/ipykernel_10232/734890896.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,employers_cnt,vacancies_cnt
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [157]:
query_5_3 = f'''
SELECT e.name employer,
       COUNT(DISTINCT v.area_id) area_cnt
FROM public.employers e
     LEFT JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY e.name
ORDER BY area_cnt DESC
'''

In [158]:
employers_area = pd.read_sql_query(query_5_3, connection)
display(employers_area)

/tmp/ipykernel_10232/57003849.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer,area_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


Работодатели с наибольшим количеством регионов, в которых они публикуют свои вакансии:

In [159]:
top_employers_area = 10 # number of employers with max areas

fig = px.bar(
    employers_area,
    x=employers_area['employer'][:top_employers_area],
    y=employers_area['area_cnt'][:top_employers_area],
    orientation='v',
    height=500,
    width=1000,
    title=f'Топ-{top_employers_area} работодателей по числу регионов, в которых они публикуют вакансии'
)
fig.update_yaxes(minor_showgrid=True)
fig.layout.xaxis.title.text = 'Работодатель'
fig.layout.yaxis.title.text = 'Количество регионов'
#fig.show()
fig.write_html('images/top_employers_area.html')
fig.write_image('images/top_employers_area.png')

![Top employers by areas](./images/top_employers_area.png)

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [160]:
query_5_4 = f'''
SELECT COUNT(e.id) employer_cnt
FROM public.employers e
     LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''

In [161]:
employers_null_industries = pd.read_sql_query(query_5_4, connection)
display(employers_null_industries)

/tmp/ipykernel_10232/1356876889.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer_cnt
0,8419


Доля работодателей в процентах, у которых не указана сфера деятельности:

In [162]:
round(employers_null_industries['employer_cnt'][0] / employers_cnt['employers_cnt'][0] * 100, 1)

35.8

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [163]:
place = 3

query_5_5 = f'''
SELECT e.name company,
       COUNT(ei.industry_id) industry_cnt
FROM public.employers e
     LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
GROUP BY e.name
HAVING COUNT(ei.industry_id) = 4
ORDER BY company
OFFSET {place - 1}
LIMIT 1
'''

In [164]:
company_four_ind = pd.read_sql_query(query_5_5, connection)
display(company_four_ind)

/tmp/ipykernel_10232/667559453.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,company,industry_cnt
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [165]:
query_5_6 = f'''
SELECT i.name industry,
       COUNT(ei.employer_id) employer_cnt   
FROM public.employers_industries ei
     JOIN public.industries i ON ei.industry_id = i.id
GROUP BY i.name
HAVING i.name = 'Разработка программного обеспечения'
'''

In [166]:
employers_software = pd.read_sql_query(query_5_6, connection)
display(employers_software)

/tmp/ipykernel_10232/361751905.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,industry,employer_cnt
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [167]:
import urllib.parse as req
# Extract table from the site then convert it to tuple
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  \
    req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
mill_cities = tuple(pd.read_html(url)[1]['Город'])

In [168]:
# Add 'Total' record using subqueries
query_5_7 = f'''
with t1 as (SELECT a.name,
                   COUNT(v.id) cnt  
            FROM public.vacancies v
                 JOIN public.employers e ON v.employer_id = e.id
                 JOIN public.areas a ON v.area_id = a.id
            WHERE e.name = 'Яндекс'
            GROUP BY a.name
            HAVING a.name in {mill_cities}
            ORDER BY cnt
            )

(SELECT *
 FROM t1
)

UNION ALL

(SELECT 'Total',
        SUM(t1.cnt)
FROM t1
)
'''

In [169]:
yandex_areas = pd.read_sql_query(query_5_7, connection)
yandex_areas['cnt'] = yandex_areas['cnt'].astype('int64')
display(yandex_areas)

/tmp/ipykernel_10232/831642525.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


График вакансий Yandex в городах миллионниках:

In [170]:
# the same operations but using window function in order to get percentage
query_5_7a = f'''
with t2 as (SELECT t1.area,
                   t1.cnt,
                   ROUND(100 * t1.cnt / SUM(t1.cnt) over (), 1) percentage
            FROM (SELECT a.name area,
                         COUNT(v.id) cnt  
                  FROM public.vacancies v
                       JOIN public.employers e ON v.employer_id = e.id
                       JOIN public.areas a ON v.area_id = a.id
                  WHERE e.name = 'Яндекс'
                  GROUP BY a.name
                  ORDER BY cnt
                 ) as t1
           )

(SELECT *
 FROM t2
 WHERE t2.area IN {mill_cities}
)

UNION ALL

(SELECT 'Total',
        SUM(t2.cnt) cnt,
        SUM(t2.percentage) percentage
FROM t2
WHERE t2.area IN {mill_cities}
)
'''

In [171]:
yandex_areas_percentage = pd.read_sql_query(query_5_7a, connection)
yandex_areas_percentage['cnt'] = yandex_areas_percentage['cnt'].astype('int64')
display(yandex_areas_percentage)

/tmp/ipykernel_10232/4005436258.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt,percentage
0,Омск,21,1.1
1,Челябинск,22,1.1
2,Красноярск,23,1.2
3,Волгоград,24,1.2
4,Пермь,25,1.3
5,Казань,25,1.3
6,Ростов-на-Дону,25,1.3
7,Уфа,26,1.3
8,Самара,26,1.3
9,Краснодар,30,1.6


In [172]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(x=yandex_areas_percentage['area'][:16], 
           y=yandex_areas_percentage['percentage'][:16], 
           marker_color="MediumPurple",
           name="Процент"),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=yandex_areas_percentage['area'][:16], 
           y=yandex_areas_percentage['cnt'][:16], 
           marker_color="MediumPurple",
           name="Число"),
    secondary_y=True,
)

fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    title_text="Вакансии Yandex в городах-миллионниках"
)

fig.update_xaxes(title_text="Город")

fig.update_yaxes(title_text="Процент вакансий", secondary_y=False)
fig.update_yaxes(title_text="Количество вакансий", secondary_y=True)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/yandex_millionaires.html')
fig.write_image('images/yandex_millionaires.png')

![Yandex cities vacancies](./images/yandex_millionaires.png)

***

Количество сфер деятельности у работодателей:

In [173]:
query_5_extra1 = f'''
SELECT e.name employer,
       COUNT(ei.industry_id) industry_cnt
FROM employers e
     LEFT JOIN employers_industries ei ON e.id = ei.employer_id
GROUP BY e.name
ORDER BY industry_cnt DESC, employer
'''

In [174]:
industry_df = pd.read_sql_query(query_5_extra1, connection)
display(industry_df)

/tmp/ipykernel_10232/1365313331.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer,industry_cnt
0,Модуль,16
1,Простые решения,11
2,Прайд,10
3,Энергия,10
4,Альфа,9
...,...,...
23170,Ярославский завод упаковочных материалов,0
23171,Ярославский филиал ПГУПС,0
23172,Ярославский ЦГМС - филиал ФГБУ Центральное УГМС,0
23173,Ярпож Казань,0


In [175]:
fig1 = px.histogram(
    industry_df,
    x='industry_cnt',
    nbins=17,
    histnorm='percent',
    height=600,
    width=1000,
    title="Распределение работодателей по числу сфер их деятельности"
)
fig1.update_layout(showlegend=False)
fig1.update_yaxes(minor_showgrid=True)
fig1.layout.xaxis.title.text = 'Количество сфер деятельности'
fig1.layout.yaxis.title.text = 'Процент работодателей'
fig1.show()

Количество работодателей в каждой сфере деятельности:

In [176]:
query_5_extra2 = f'''
SELECT ine.industry,
       ine.employer_cnt,
       ROUND(100 * ine.employer_cnt / SUM(ine.employer_cnt) over (), 1) employer_percentage
FROM (
     SELECT i.name industry,
            COUNT(ei.employer_id) employer_cnt
     FROM industries i
          LEFT JOIN employers_industries ei ON i.id = ei.industry_id
     GROUP BY i.name
     ORDER BY employer_cnt DESC, industry
     ) as ine
'''

In [177]:
employers_df = pd.read_sql_query(query_5_extra2, connection)
display(employers_df)

/tmp/ipykernel_10232/4210231535.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,industry,employer_cnt,employer_percentage
0,Разработка программного обеспечения,3553,11.0
1,"Системная интеграция, автоматизации технологи...",2993,9.3
2,"Интернет-компания (поисковики, платежные систе...",1675,5.2
3,"Маркетинговые, рекламные, BTL, дизайнерские, E...",798,2.5
4,Консалтинговые услуги,662,2.0
...,...,...,...
289,"Лифтовое хозяйство (монтаж, сервис, ремонт)",4,0.0
290,Ритуальные услуги,4,0.0
291,Автошкола,3,0.0
292,"Ботанический сад, зоопарк, заповедник",3,0.0


In [178]:
# Reduce industry names to three words
employers_df['industry'] = employers_df['industry'].apply(lambda x: ' '.join(x.split()[:3]) + ' ...' \
    if len(x.split()) > 3 else x)

top_industry = 5

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(x=employers_df['industry'][:top_industry], 
           y=employers_df['employer_percentage'][:top_industry], 
           marker_color="MediumPurple",
           name="Процент"),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=employers_df['industry'][:top_industry], 
           y=employers_df['employer_cnt'][:top_industry], 
           marker_color="MediumPurple",
           name="Число"),
    secondary_y=True,
)

fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    title_text=f"Топ-{top_industry} отраслей по числу работодателей, занятых в них"
)

fig.update_xaxes(title_text="Отрасль")

fig.update_yaxes(title_text="Процент работодателей", secondary_y=False)
fig.update_yaxes(title_text="Количество работодателей", secondary_y=True)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/top_industry_employers.html')
fig.write_image('images/top_industry_employers.png')

![Top industry by employers](./images/top_industry_employers.png)

Количество сфер деятельности, в которые вовлечены работодатели занимающиеся разработкой программного обеспечения:

In [179]:
query_5_extra3 = f'''
SELECT e.name employer,
       COUNT(ei.industry_id) industry_cnt
FROM employers e
     LEFT JOIN employers_industries ei ON e.id = ei.employer_id
     LEFT JOIN industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
GROUP BY e.name
ORDER BY industry_cnt DESC, employer
'''

In [180]:
developer_df = pd.read_sql_query(query_5_extra3, connection)
display(developer_df)

/tmp/ipykernel_10232/3618769863.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer,industry_cnt
0,Константа,3
1,Простые решения,3
2,Эксперт,3
3,Admitad,2
4,EFSOL,2
...,...,...
3530,Языковой центр Биг Бен,1
3531,Ямальский расчетно-информационный центр,1
3532,ЯрНео,1
3533,Яроцкий Е. В./ MoonX,1


Распределение работодателей, занимающихся разработкой программного обеспечения, по числу отраслей, в которых они заняты:

In [181]:
query_5_extra4 = f'''
with industry_count_table as (
    SELECT e.name employer,
       COUNT(ei.industry_id) industry_cnt
FROM employers e
     LEFT JOIN employers_industries ei ON e.id = ei.employer_id
     LEFT JOIN industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
GROUP BY e.name
ORDER BY industry_cnt DESC, employer
)

SELECT ict.industry_cnt industry_cnt,
       COUNT(ict.employer) employer_cnt
FROM industry_count_table as ict
GROUP BY ict.industry_cnt
ORDER BY industry_cnt DESC
'''

In [182]:
developers_industries_df = pd.read_sql_query(query_5_extra4, connection)
display(developers_industries_df)

/tmp/ipykernel_10232/3277535228.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,industry_cnt,employer_cnt
0,3,3
1,2,12
2,1,3520


***

*В пятёрке лидеров по общему вакансий Yandex лидирует с большим отрывом: его вакансии занимают чуть менее $4\%$ от общего количества. Далее находятся телекоммуникационная компания (Ростелеком), два крупных банка (Сбер, Тинькофф) и одно сырьевое предприятие (Газпром Нефть). Таким образом, среди лидеров по данному критерию представлены достаточно крупные компании, что неудивительно. Но даже несмотря на это, Yandex занимает всего лишь несколько процентов от общего числа вакансий, т.е. остальные распределены в том числе между компаниями поменьше.*

*В базе присутствует лишь один регион, в котором нет вакансий, но зарегистрированы работодатели: Россия. Это можно объяснить тем, что вакансии работодателя присутствуют во многих регионах, а точный юридический адрес указывать в вакансии не имеет никакого смысла.*

*Среди работодателей с наибольшим числом регионов, в которых их вакансии опубликованы, также лидирует Yandex. Его вакансии размещены в 181 регионе. В целом, среди топ-10 работодателей по данному критерию можно встретить, как крупные компании (МТС, Почта России), так и малоизвестные. Вполне вероятно, что связано со спецификой их сферы деятельности.*

*Чуть меньше, чем $36\%$ работодателей не указывают сферу деятельности. Среди возможных причин такого явления можно выделить: слишком большое количество сфер деятельности, сфера деятельности не подходит ни под одну из предлагаемых, игнорирование.*

*Примерно четверть всех вакансий Yandex размещена в городах-миллионниках. Из них можно выделить Москву с $2.8\%$, Санкт-Петербург - $2.2\%$ и Екатеринбург - $2\%$. По сравнению с ситуацией на рынке в целом вакансии Yandex распределены более равномерно. Это можно объяснить спецификой сферы деятельности компании.*

*Среди тех работодателей, которые указали сферу деятельности, наблюдается обратная зависимость количества работодателей относительно числа сфер деятельности, в которых они вовлечены: большим числом отраслей, занято меньшее число компаний. Например, одну сферу деятельности имеют около $27.5\%$ работодателей. Наибольшее количество сфер деятельности 16 - имеет лишь один работодатель. Такое распределение выглядит естественным: большое количество сфер деятельности, как правило, подразумевает под собой большой штат сотрудников. Таких компаний просто физически меньше.*

*По количеству работодателей пять лидирующих сфер деятельности выглядит следующим образом: разработка программного обеспечения ($11\%$), системная интеграция (более $9\%$), интернет-компании (более $5\%$), маркетинг ($2.5\%$), консалтинг ($2\%$). Подобное распределение можно объяснить как и спецификой платформы (базы данных), так и отсутствием доминирующих крупных фирм в этих сферах деятельности.*

*Среди работодателей, вовлечённых в разработку программного обеспечения, число фирм, которые задействованы в других отраслях крайне мало (15 работодателей). Такое явление можно объяснить тем, что в этой сфере преобладают небольшие компании с малым штатом сотрудников.*

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [183]:
# Filter query with ILIKE operators
query_6_1 = f'''
SELECT COUNT(v.id) vacancies_cnt
FROM public.vacancies v
WHERE v.name ILIKE '%data%' OR v.name ILIKE '%данн%'
'''

In [184]:
data_vacancies = pd.read_sql_query(query_6_1, connection)
display(data_vacancies)

/tmp/ipykernel_10232/1514950384.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vacancies_cnt
0,1771


Доля от общего количества вакансий в процентах:

In [185]:
round(data_vacancies['vacancies_cnt'][0] / vacancies['vacancies_cnt'][0] * 100, 1)

3.6

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [186]:
# Combine filters
query_6_2 = f'''
SELECT COUNT(v.id) vacancies_cnt
FROM public.vacancies v
WHERE 
    (
        (
            v.name ILIKE '%data science%' OR 
            v.name ILIKE '%data scientist%' OR 
            v.name ILIKE '%исследователь данных%' OR 
            v.name ILIKE '%машинн%обучен%' OR 
            v.name ILIKE '%machine learning%' OR 
            v.name LIKE '%ML%'
        ) 
        AND NOT (v.name ILIKE '%html%')
    )
    AND (
            v.name ILIKE '%junior%' OR 
            v.experience ILIKE '%нет опыта%' OR 
            v.employment ILIKE '%стажировка%'
        )
'''

In [187]:
data_vacancies_junior = pd.read_sql_query(query_6_2, connection)
display(data_vacancies_junior)

/tmp/ipykernel_10232/1192317152.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vacancies_cnt
0,51


Доля вакансий для начинающего DS в процентах от числа всех вакансий по DS:

In [188]:
query_6_2a = f'''
SELECT COUNT(v.id) vacancies_cnt
FROM public.vacancies v
WHERE 
    (
        (
            v.name ILIKE '%data science%' OR 
            v.name ILIKE '%data scientist%' OR 
            v.name ILIKE '%исследователь данных%' OR 
            v.name ILIKE '%машинн%обучен%' OR 
            v.name ILIKE '%machine learning%' OR 
            v.name LIKE '%ML%'
        ) 
        AND NOT (v.name ILIKE '%html%')
    )
'''

ds_vacancies= pd.read_sql_query(query_6_2a, connection)
display(ds_vacancies)

/tmp/ipykernel_10232/2228532801.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vacancies_cnt
0,480


In [189]:
round(data_vacancies_junior['vacancies_cnt'][0] / ds_vacancies['vacancies_cnt'][0] * 100, 1)

10.6

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

*Критерии для отнесения вакансии к DS указаны в предыдущем задании.**

In [190]:
query_6_3 = f'''
SELECT COUNT(v.id) vacancies_cnt
FROM public.vacancies v
WHERE 
    (
        (
            v.name ILIKE '%data science%' OR 
            v.name ILIKE '%data scientist%' OR 
            v.name ILIKE '%исследователь данных%' OR 
            v.name ILIKE '%машинн%обучен%' OR 
            v.name ILIKE '%machine learning%' OR 
            v.name LIKE '%ML%'
        ) 
        AND NOT (v.name ILIKE '%html%')
    )
    AND
        (v.key_skills ILIKE '%SQL%' OR v.key_skills LIKE '%postgres%')
'''

In [191]:
ds_keyskills = pd.read_sql_query(query_6_3, connection)
display(ds_keyskills)

/tmp/ipykernel_10232/1355946866.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vacancies_cnt
0,201


Доля вакансий со знанием Postgres или SQL в ключевых навыках в процентах от числа всех вакансий по DS:

In [192]:
round(ds_keyskills['vacancies_cnt'][0] / ds_vacancies['vacancies_cnt'][0] * 100, 1)

41.9

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

*Это можно сделать помощью запроса, аналогичного предыдущему.*

In [193]:
query_6_4 = f'''
SELECT COUNT(v.id) vacancies_cnt
FROM public.vacancies v
WHERE 
    (
        (
            v.name ILIKE '%data science%' OR 
            v.name ILIKE '%data scientist%' OR 
            v.name ILIKE '%исследователь данных%' OR 
            v.name ILIKE '%машинн%обучен%' OR 
            v.name ILIKE '%machine learning%' OR 
            v.name LIKE '%ML%'
        ) 
        AND NOT (v.name ILIKE '%html%')
    )
    AND
        (v.key_skills ILIKE '%python%')
'''

In [194]:
ds_python = pd.read_sql_query(query_6_4, connection)
display(ds_python)

/tmp/ipykernel_10232/2699308984.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vacancies_cnt
0,351


Доля вакансий с владением Python в ключевых навыках в процентах от числа всех вакансий по DS:

In [195]:
round(ds_python['vacancies_cnt'][0] / ds_vacancies['vacancies_cnt'][0] * 100, 1)

73.1

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [196]:
# remove tabulations via replace function then calculation difference between two lengths
# in order to get number of tabulation signs
query_6_5 = f'''
SELECT ROUND(AVG(length(v.key_skills) - length(replace(v.key_skills, CHR(9), '')) + 1), 2) avg_skill_cnt
FROM public.vacancies v
WHERE 
    (
        (
            v.name ILIKE '%data science%' OR 
            v.name ILIKE '%data scientist%' OR 
            v.name ILIKE '%исследователь данных%' OR 
            v.name ILIKE '%машинн%обучен%' OR 
            v.name ILIKE '%machine learning%' OR 
            v.name LIKE '%ML%'
        ) 
        AND NOT (v.name ILIKE '%html%')
        AND v.key_skills IS NOT NULL
    )
'''

In [197]:
len_skills = pd.read_sql_query(query_6_5, connection)
display(len_skills)

/tmp/ipykernel_10232/1138377757.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,avg_skill_cnt
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [198]:
# here, coalesce function allows us to calculate mean value whether one of the values is null or not
query_6_6 = f'''
SELECT DISTINCT v.experience,
       ROUND(AVG((COALESCE(v.salary_from, v.salary_to) + COALESCE(v.salary_to, v.salary_from)) / 2)) avg_salary
FROM public.vacancies v
WHERE 
    (
        (
            v.name ILIKE '%data science%' OR 
            v.name ILIKE '%data scientist%' OR 
            v.name ILIKE '%исследователь данных%' OR 
            v.name ILIKE '%машинн%обучен%' OR 
            v.name ILIKE '%machine learning%' OR 
            v.name LIKE '%ML%'
        ) 
        AND NOT (v.name ILIKE '%html%')
        AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
    )
GROUP BY v.experience
'''

In [199]:
avg_salary = pd.read_sql_query(query_6_6, connection)
display(avg_salary)

/tmp/ipykernel_10232/2901889043.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


In [200]:
fig = px.bar(
    avg_salary,
    x=avg_salary['experience'],
    y=avg_salary['avg_salary'],
    orientation='v',
    height=500,
    width=1000,
    title='Средние зарплаты в Data Science в зависимости от опыта работы'
)
fig.update_yaxes(minor_showgrid=True)
fig.layout.xaxis.title.text = 'Опыт работы'
fig.layout.yaxis.title.text = 'Заработная плата, руб'
#fig.show()
fig.write_html('images/ds_salary_exp.html')
fig.write_image('images/ds_salary_exp.png')

![DS mean salaries](./images/ds_salary_exp.png)

***

Количество вакансий в DS в зависимости от опыта работы:

In [201]:
query_6_extra1 = f'''
SELECT v.experience,
       COUNT(v.id) vacancies_cnt
FROM public.vacancies v
WHERE 
    (
        (
            v.name ILIKE '%data science%' OR 
            v.name ILIKE '%data scientist%' OR 
            v.name ILIKE '%исследователь данных%' OR 
            v.name ILIKE '%машинн%обучен%' OR 
            v.name ILIKE '%machine learning%' OR 
            v.name LIKE '%ML%'
        ) 
        AND NOT (v.name ILIKE '%html%')
    )
GROUP BY v.experience
ORDER BY vacancies_cnt DESC
'''

In [202]:
ds_experience = pd.read_sql_query(query_6_extra1, connection)
display(ds_experience)

/tmp/ipykernel_10232/3747617107.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,vacancies_cnt
0,От 1 года до 3 лет,223
1,От 3 до 6 лет,195
2,Более 6 лет,34
3,Нет опыта,28


In [203]:
fig = px.pie(
    ds_experience,
    values=ds_experience['vacancies_cnt'],
    names=ds_experience['experience'],
    title='Распределение вакансий Data Science по опыту работы',
    height=600,
    width=1000
)
fig.update_traces(textposition='outside', textinfo='percent+label', textfont_size=14)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/ds_exp.html')
fig.write_image('images/ds_exp.png')

![DS vacancies by experience](./images/ds_exp.png)

Число вакансий в DS у различных работодателей:

In [204]:
query_6_extra2 = f'''
SELECT 
    t1.employer employer,
    t1.vacancies_cnt vacancies_cnt,
    ROUND(100 * t1.vacancies_cnt / SUM(t1.vacancies_cnt) over (), 1) percentage
FROM (
        SELECT e.name employer,
               COUNT(v.id) vacancies_cnt
        FROM public.vacancies v
        JOIN employers e ON v.employer_id = e.id
        WHERE 
        (
            (
                v.name ILIKE '%data science%' OR 
                v.name ILIKE '%data scientist%' OR 
                v.name ILIKE '%исследователь данных%' OR 
                v.name ILIKE '%машинн%обучен%' OR 
                v.name ILIKE '%machine learning%' OR 
                v.name LIKE '%ML%'
            ) 
            AND NOT (v.name ILIKE '%html%')
        )
        GROUP BY employer
        ORDER BY vacancies_cnt DESC
    ) as t1
'''

In [205]:
ds_employers = pd.read_sql_query(query_6_extra2, connection)
display(ds_employers)

/tmp/ipykernel_10232/3545300133.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employer,vacancies_cnt,percentage
0,СБЕР,37,7.7
1,Bell Integrator,25,5.2
2,Банк ВТБ (ПАО),18,3.8
3,VK,15,3.1
4,Positive Technologies,11,2.3
...,...,...,...
220,Иллекс,1,0.2
221,ИНВИТРО,1,0.2
222,Институт искусственного интеллекта AIRI,1,0.2
223,Интеллектуальные решения,1,0.2


In [206]:
top_employers = 10
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Bar(x=ds_employers['employer'][:top_employers], 
           y=ds_employers['percentage'][:top_employers], 
           marker_color="MediumPurple",
           name="Процент"),
    secondary_y=False,
)

fig.add_trace(
    go.Bar(x=ds_employers['employer'][:top_employers], 
           y=ds_employers['vacancies_cnt'][:top_employers], 
           marker_color="MediumPurple",
           name="Число"),
    secondary_y=True,
)

fig.update_layout(
    autosize=False,
    width=1000,
    height=600,
    title_text=f"Топ-{top_employers} работодателей с наибольшим числом вакансий в DS"
)

fig.update_xaxes(title_text="Работодатель")

fig.update_yaxes(title_text="Процент от числа вакансий DS", secondary_y=False)
fig.update_yaxes(title_text="Количество вакансий DS", secondary_y=True)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/top_employers_ds.html')
fig.write_image('images/top_employers_ds.png')

![Top employers by DS vacancies](./images/top_employers_ds.png)

Распределение вакансий DS по регионам:

In [207]:
top_area = 5
query_6_extra3 = f'''
with t1 as (
    SELECT a.name area,
           COUNT(v.id) vacancies_cnt
    FROM public.vacancies v
         JOIN public.areas a ON v.area_id = a.id
    WHERE 
        (
            (
                v.name ILIKE '%data science%' OR 
                v.name ILIKE '%data scientist%' OR 
                v.name ILIKE '%исследователь данных%' OR 
                v.name ILIKE '%машинн%обучен%' OR 
                v.name ILIKE '%machine learning%' OR 
                v.name LIKE '%ML%'
            ) 
            AND NOT (v.name ILIKE '%html%')
        )
    GROUP BY area
    ORDER BY vacancies_cnt DESC
    OFFSET {top_area}
)

(SELECT a.name area,
        COUNT(v.id) vacancies_cnt
 FROM public.vacancies v
         JOIN public.areas a ON v.area_id = a.id
 WHERE (
            (
                v.name ILIKE '%data science%' OR 
                v.name ILIKE '%data scientist%' OR 
                v.name ILIKE '%исследователь данных%' OR 
                v.name ILIKE '%машинн%обучен%' OR 
                v.name ILIKE '%machine learning%' OR 
                v.name LIKE '%ML%'
            ) 
            AND NOT (v.name ILIKE '%html%')
        )
 GROUP BY area
 ORDER BY vacancies_cnt DESC
 LIMIT {top_area} 
)

UNION ALL

(SELECT 
    'Другие',
    SUM(t1.vacancies_cnt)
 FROM t1
)
'''

In [208]:
ds_area = pd.read_sql_query(query_6_extra3, connection)
ds_area['vacancies_cnt'] = ds_area['vacancies_cnt'].astype('int64')
display(ds_area)

/tmp/ipykernel_10232/3050899939.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,vacancies_cnt
0,Москва,217
1,Санкт-Петербург,64
2,Новосибирск,23
3,Нижний Новгород,20
4,Казань,16
5,Другие,140


In [209]:
fig = px.pie(
    ds_area,
    values=ds_area['vacancies_cnt'],
    names=ds_area['area'],
    title=f'Топ-{top_area} регионов по числу вакансий DS',
    height=600,
    width=1000
)
fig.update_traces(textposition='outside', textinfo='percent+label', textfont_size=14)
fig.update(layout_showlegend=False)
#fig.show()
fig.write_html('images/top_area_ds.html')
fig.write_image('images/top_area_ds.png')

![Top areas by DS vacancies](./images/top_area_ds.png)

***

*Среди общего числа вакансий около $3.6\%$ можно каким-либо образом отнести к работе с данными.*

*Более $10\%$ вакансий Data Science можно отнести к вакансиям для начинающих специалистов, либо для соискателей без опыта.*

*Около $42\%$ вакансий Data Science требуют владение SQL, что подчёркивает важность владения языком запросов к базам данных в Data Science. Базы данных - основной метод хранения данных.*

*Более $73\%$ вакансий Data Science требуют от соискателя знания Python. Python - основной язык в Data Science, поэтому эта информация соответствует ожиданиям.*

*Среднее количество требуемых ключевых навыков для Data Science $6.41$. Даже если учесть, что часть из них - это soft skills, получается, что эта профессия требует владения широким набором инструментов.*

*Заработные платы в Data Science выше, чем в среднем по рынку и показывают хорошую прогрессию в зависимости от опыта работы.*

*В сравнении с общим спросом на соискателей в зависимости от их опыта работы Data Science имеет серьёзный крен в сторону более опытных работников. Чуть менее $41\%$ всех вакансий для стажа 3-6 лет и около $7\%$ для опыта работы более 6 лет против $29.5\%$ и $2.7\%$ соответственно для вакансий в среднем по базе. Такое смещение можно объяснить следующим образом: возможно, в Data Science даже при большом стаже продолжается улучшение навыков, уровень владения новыми инструментами для достижения профессиональных целей.*

*Что касается распределения вакансий, то мы наблюдаем лидеров: СБЕР - $7.7\%$, Bell Integrator - $5.2\%$, ВТБ - $3.8\%$ и затем плавный спад. С учётом того, что у лидеров небольшой процент от общего числа вакансий, то можно заключить, что Data Science распространён среди различных сфер деятельности.*

*Рассматривая распределение вакансий Data Science по регионам, можно прийти к выводу, что вакансии значительно более централизованны, нежели вакансии в среднем по базе данных: Москва $45.2\%$ против $10.8\%$. Это можно объяснить тем, что в крупных городах наблюдается более высокая концентрация как высококвалифицированных кадров, так и необходимой инфраструктуры для высокотехнологичных отраслей.*

# Общий вывод по проекту

*По итогам исследования можно предположить, что в узкоспециализированных высококвалицированных отраслях выше спрос на кадры с большим опытом работы, подобные вакансии сконцентрированы в крупных городах и имеют более высокую заработную плату.*

*Что касается Data Science, то ему присущи все вышеописанные свойства. Добавить можно то, что вакансии более равномерны распределены по сферам деятельности, так как данные есть везде, и есть потребность их анализировать.*

In [210]:
connection.close()